In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.naive_bayes import GaussianNB

In [11]:
df = pd.read_excel('spalling.xlsx', index_col=0)

In [16]:
df.head()

,spalling,time,weightloss,firecurve,load,stress,Age,moisture,Compressive strength
name,,,,,,,,,
1.1,20,14.8,8,std,624,6.24,176,4.5,63
1.4,21,14.8,9,std,617,6.17,177,4.5,63
1.5,17,12.7,3,std,634,6.34,400,4.5,63
2.1,32,9,19,std,622,6.22,180,4.1,61
2.4,19,12.8,10,std,616,6.16,180,4.1,61


In [14]:
# df_spall = df.loc[df['spalling']!=0]
df_spall = df.loc[df['firecurve']=='std']
df_spall = df_spall.drop('firecurve', axis=1)
df_spall = df_spall.drop('weightloss', axis=1)
df_spall = df_spall.drop('time', axis=1)
df_spall = df_spall.drop('Age', axis=1)
df_use = df_spall

In [72]:
df_use.head(5)

,spalling,load,stress,moisture,Compressive strength
name,,,,,
1.1,1,624,6.24,4.5,63
1.4,1,617,6.17,4.5,63
1.5,1,634,6.34,4.5,63
2.1,1,622,6.22,4.1,61
2.4,1,616,6.16,4.1,61


In [82]:
def convert(number):
    if number > 0:
        return 1
    else:
        return 0

In [119]:
df_use.drop(df_use[df_use.moisture == '-'].index, inplace=True)
df_use.head(5)

,spalling,load,stress,moisture,Compressive strength
name,,,,,
1.1,1,624,6.24,4.5,63
1.4,1,617,6.17,4.5,63
1.5,1,634,6.34,4.5,63
2.1,1,622,6.22,4.1,61
2.4,1,616,6.16,4.1,61


In [88]:
df_use['spalling'] = df_use['spalling'].apply(lambda x:convert(x))

In [108]:
data = df_use.drop('spalling',axis=1)
target = df_use['spalling']

In [125]:
def concrete_classifier(data, target):
    X, y = data, target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8)
    
    gaus_model = GaussianNB()    
    gaus_model.fit(X_train,y_train) 
    
    return gaus_model

In [126]:
result_model = concrete_classifier(data,target)

In [134]:
result_model.predict_proba([data.iloc[7,:]])[0][1]

0.4787934784238976

In [132]:
int(result_model.predict([data.iloc[10,:]]))

0